# Race Simulation

This notebook loads the best-performing model and simulates a race using drivers' skills, weather data, and practice data.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Load the Best Model

In [ ]:
# Load the best model (assuming Random Forest performed best)
best_model = joblib.load('Random_Forest_model.pkl')

# Load the preprocessor
preprocessor = joblib.load('preprocessor.pkl')  # Save this during training

## Define Race Simulation Functions

In [ ]:
def simulate_race(drivers, total_laps, weather_conditions):
    race_results = []
    for lap in range(1, total_laps + 1):
        lap_data = []
        for driver in drivers:
            # Update dynamic features
            driver['tire_age'] += 1
            driver['fuel_load'] -= 1.5  # Example consumption per lap
            
            # Prepare input features
            features = pd.DataFrame({
                'driver_overall_skill': [driver['driver_overall_skill']],
                'driver_circuit_skill': [driver['driver_circuit_skill']],
                'driver_consistency': [driver['driver_consistency']],
                'driver_reliability': [driver['driver_reliability']],
                'driver_aggression': [driver['driver_aggression']],
                'driver_risk_taking': [driver['driver_risk_taking']],
                'fp1_median_time': [driver['fp1_median_time']],
                'fp2_median_time': [driver['fp2_median_time']],
                'fp3_median_time': [driver['fp3_median_time']],
                'quali_time': [driver['quali_time']],
                'tire_age': [driver['tire_age']],
                'fuel_load': [driver['fuel_load']],
                'track_position': [driver['track_position']],
                'track_temp': [weather_conditions['track_temp']],
                'air_temp': [weather_conditions['air_temp']],
                'humidity': [weather_conditions['humidity']],
                'tire_compound': [driver['tire_compound']],
                'TrackStatus': [weather_conditions['TrackStatus']],
                'is_pit_lap': [0]
            })
            
            # Transform features
            X = preprocessor.transform(features)
            
            # Predict lap time
            lap_time = best_model.predict(X)[0]
            
            # Store lap data
            lap_data.append({'driver': driver['name'], 'lap_time': lap_time})
        
        # Update positions based on lap times
        lap_data.sort(key=lambda x: x['lap_time'])
        for position, data in enumerate(lap_data, start=1):
            for driver in drivers:
                if driver['name'] == data['driver']:
                    driver['track_position'] = position
                    break
        
        # Store race results
        race_results.append(lap_data)
    return race_results

## Simulate the Race

In [ ]:
# Define drivers
drivers = [
    {
        'name': 'Driver A',
        'driver_overall_skill': 0.85,
        'driver_circuit_skill': 0.80,
        'driver_consistency': 0.75,
        'driver_reliability': 0.90,
        'driver_aggression': 0.60,
        'driver_risk_taking': 0.55,
        'fp1_median_time': 88000,
        'fp2_median_time': 87500,
        'fp3_median_time': 87000,
        'quali_time': 86000,
        'tire_age': 0,
        'fuel_load': 100.0,
        'track_position': 1,
        'tire_compound': 1  # Hard
    },
    # Add more drivers as needed
]

# Define weather conditions
weather_conditions = {
    'track_temp': 35.0,
    'air_temp': 25.0,
    'humidity': 50.0,
    'TrackStatus': 1
}

# Simulate the race
total_laps = 50
race_results = simulate_race(drivers, total_laps, weather_conditions)

## Analyze Results

In [ ]:
# Convert race results to DataFrame
race_df = pd.DataFrame()
for lap_num, lap_data in enumerate(race_results, start=1):
    for data in lap_data:
        race_df = race_df.append({
            'Lap': lap_num,
            'Driver': data['driver'],
            'LapTime': data['lap_time']
        }, ignore_index=True)

# Display race DataFrame
race_df.head()

## Plot Lap Times

In [ ]:
# Plot lap times for each driver
plt.figure(figsize=(12, 6))
for driver in race_df['Driver'].unique():
    driver_data = race_df[race_df['Driver'] == driver]
    plt.plot(driver_data['Lap'], driver_data['LapTime'], label=driver)
plt.xlabel('Lap')
plt.ylabel('Lap Time (ms)')
plt.title('Lap Times by Driver')
plt.legend()
plt.show()